$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$\newcommand{\inp}[2]{\langle{#1}|{#2}\rangle}$ 
$\newcommand{\bracket}[2]{\langle{#1}|{#2}\rangle}$
$\newcommand{\ds}{\displaystyle}$
$\newcommand{\id}{\mathbb{ID}}$
$\newcommand{\tensor}{\otimes}$
$\newcommand{\hide}[1]{}$
# One-out-of-two Quantum Oblivious Transfer based on Nonorthogonal States

What is an obivious transfer (OT) protocol?
An oblivious transfer protocol means a sender send a or more pieces of information and stays oblivious if the message which message was chosen.

Most common protocols are all-or-nothing and one-out-of-two. <br>
All-or-nothing OT (1981):<br>
A all-or-nothing protocol is when Bob learns the message sent by Alice either with 100% reliability, or 0%. So Alice sends a message $m \in {0,1}$ to Bob. Bob receives the message m wirh a porbability of 50%. Bob either learns *m* with 100% reliability or 0% and Alice does not know if Bob received the message. This is oblivious transfer, because  Bob either gets the correct piece of information or gets no useful information at all, and Alice is unaware of whether Bob has successfully received the message.

One-out-of-two OT (1-2 OT): <br>
Alice send two messages to Bob, $m_0$ and $m_1$. Bob chooses to learn one message, and learns nothing about the other message, and Bob does not disclose his choice to Alice. This is oblivious transfer because Alice never learns what message Bob knows, and Bob does not learn any of the content of the message he did not choose.

Build 1-2 OT using p-all-or-nothing OT: <br>
Bob generate two keys, $key_1, key_2$, and uses the all-or-nothing protocol to learn key with 100% certainty, the other with 0%. Bob asks Alice to encrypt mesages $m_0$ and $m_1$ with using $key_j$ with $j \in {0,1}$, so the key he chose to learn. Bob receives the messages and can only decrypt the message for which he chose the hey. An example protocol:
- step 1: Bob Generates Keys
- step 2: All-or-Nothing OT for Keys
- step 3: Alice Encrypts Messages
- step 4: Transmission of Ciphertexts
- step 5: Bob Decrypts the Message <br>

This is a two-level structured protocol

Previous one-out-of two quantum oblivious transfer (QOT) scheme have a two-level structure based in no-go theorem.

The no-go theorem:<br>
in 1997, Lo introduced the no-go theorem for OT protocols. The theorem says that all one-sided two-party computations are potentialy insecure. It applies to all protocols where two parites must input two values i and j together into a fuction f(i,j), and only one party is allowed to learn the result. This theroem contains the one-out-of-two Quantum Oblivious Transfer (QOT) where the function f(i,j) is the selector. <br>
The theorem means to show the security holes of OT protocols by demonstrating that the party not learning the result could potentially gain information.

To counteract this theorem He's proof was created. With the proof new oblivious transfer protocols can be created.

He's proof: <br>
In the proof the creation of a secure all-or-nothing QOT scheme is done that uses four entangled states, and avoids the limitations defined by Lo’s no-go theorem. The key to success of the proof is that the receiver inputs hish choices before the sender inputs her messages $m_0$ and $m_1$. By doing this it allows to differentiate the function f of the one-out-of-two protocol and those from Lo’s no-go theory. This led to the development of other QOT protocols that are not constrained by Lo's theorem, expanding the possibilities for secure quantum communication in cryptography.

A new protocol unsing this proof is proposed in the next section.

### Proposed Protocol

The new protocol proposed in the parper has 7 steps:

Step 1: <br>
- Bob creates qubit sequence according to his intentions
    - $j_0$, choose $\ket 0$ 
    - $j_1$, choose $\ket +$
- Sequence must be longer than OT sequence
- N is minimum length (Bob receives N messages), M number channel checking qubits, K number loyalty testing qubits
    - $QOT = N \in (\ket0, \ket+) + M \in (\ket0,\ket1,\ket+,\ket-) + 2K \in (\ket0,\ket+) $
- Send sequence to Alice

> Example: <br>
Two qubits: $\ket{0+}_{12} (N)$ to represent his choices and $\ket{0+}_{34} (2K)$ for loyalty testing


Step 2:

- Alice receives sequence: checks for eavesdroppers and tests Bobs loyalty
1. asks Bob to publish bases and states he created, <br>
if error rate higher than threshold, then Eve is present on	channel 
    - communication aborted
2. Checks Bobs loyalty: Alice randomly selects several positions and requests Bob to publish his bases
    - If different results (e.g. not in $\ket{0}$,$\ket{+}$), measured is higher than given error rate then Bob is dishonest
    - Communication aborted

> $\ket {0+0+}_{1234}$ , suppose Alice chooses qubit 4, asks Bob to publish base of qubit 4, measures it and compares the published and measured result. <br>
Afterwards qubit 4 is discarded.


Step 3:

- Because of the loyalty test Bob must ask Alice to reorder qubits
    - This is done as a precaution, in case eavesdropper eve was able to catch the message, the message is reordered so eve doesn't know the message going forward
- This is what the additional 2K qubits are for, prevent vacancies in the lost of choice intentions
- The sequence after reordering represents Bobs real choices

> Bobs asks Alice to reorder remaining qubits in order 21, <br>
States become $\ket{+0}$, with qubit 3 discarded, <br>
Represent Bob’s choices $j_0$ and $j_1$ respectively


Step 4:

- Alice inputs secret message m0 and m1 through I,X,Y,Z operations according to 00,10,11,01

> Alice performs Z and X so $01_{12}$ and $10_{34} $ <br>
Converts $\ket{+0}_{21}$ to $\ket{-1}_{21}$


Step 5:
- Alice inserts decoy qubits from $\ket0,\ket1,\ket+,\ket-$ for channel checking 
- Send sequence to Bob


Step 6:

- Bob receives sequence and asks Alice to publish positions and states of decoy qubits
- If error rate higher than channel error rate abort communication
    - Return to step 1
- Otherwise Bob learns contents of the classical message by measuring the qubits with bases he prepared

> Bob performs X and Z basis measurements to learn second and first classical messages, 1 and 1 <br>
($01_{12}$ and $10_{34}$)


Step 7:
- Bob test Alice loyalty
- Bob chooses random positions and asks Alice to publish her operations
- Bob performs operations according to announcements to recover qubits into   $\{\ket0,\ket+\}$
- Check error rate, if higher than threshold then Alice is dishonest

> Bob asks Alice to publish operations (X) which performs on qubit 1 <br>
Bob performs X on qubit 1 to recover state $\ket0$


Why is this an oblivious transfer protocol? <br>
This is an oblivious transfer protocolm because Bob only learns either message $m_0$ or message $m_1$, and nothing about the other message, and Alice does not know which message bob chooses to learn. <br>
In step 1 Bob disguises his choices in step 1. This step also helps with the bindingness component that are expected for cyptographic protocols. In step 4 the bits are reordered, so Bob does not know which qubits belong to message 0, and which to message 1. Bob chooses a message $m_0$ or $m_1$ without knowing which message he is choosing. How Bob knows which qubit belong to which message, is because Bob asks Alice to publish position and states in step 7. As Alice gives Bob the states and position in step 7, but does not learn which message he chooses in the end, Alice never learns which message he chose to learn. 

In [ ]:
#Implementation in Qiskit

from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
#from qiskit_textbook.tools import simon_oracle

# Step 1: Bob creates qubit sequence
# Define qubit sequence length
# EXAMPLE:
N = 2  # Bob's qubits
M = 2  # channel checking qubits
K = 2  # loyalty testing qubits

# Create a quantum circuit with N + M + 2K qubits
qc = QuantumCircuit(N + M + 2 * K)

# Bob prepares qubits for choice intentions (|0〉 and |+〉 states)
qc.h(range(N))  # |+〉 state for choice intention j1
# Insert qubits for channel checking and loyalty testing
qc.reset(range(N, N + M + 2 * K))  # Initialize qubits for channel checking and loyalty testing

# Step 2: 

# Step 3: 

# Step 4: 

# Step 5: 

# Step 6: 

# Step 7: 

# Execute the quantum circuit
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(qc).result()
counts = result.get_counts()

# Visualize the measurement outcomes
print(counts)
plot_histogram(counts)


The implementation is dependent on the problem, the implementation of the example:

In [122]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram


# Step 1: Bob creates a qubit sequence
qc_bob = QuantumCircuit(4, 2)  # 4 qubits, 2 classical bits for measurement
qc_bob.h(1)  # Choice intention |+> for j1 (qubit 1)
#qc_bob.h(3)  # Loyalty testing |+> (qubit 3)

# Step 2: Alice checks the channel and Bob's loyalty
#qc_bob.measure(3, 0)  # Measure qubit 4 (index 3) for loyalty

# Simulate the loyalty check

#print("Loyalty check results (qubit 4):", counts)

# Assuming loyalty check passed, we continue
qc_bob.reset(3)  # Reset qubit 4 for reuse

# Step 3: Reorder qubits (keeping original indices for clarity)
qc_bob.reset(0)
qc_bob.reset(1)
qc_bob.h(0)

# Step 4: Alice performs Z and X operations to encode messages
qc_bob.z(0)  # Applying Z to qubit 0 (to encode message '0')
qc_bob.x(1)  # Applying X to qubit 1 (to encode message '1')


# Step 5: Alice inserts decoy qubits


# Step 6: Bob measures the qubits to learn the classical messages
qc_bob.measure(0,1)
#qc_bob.measure([0,1],[0,0])
#qc_bob.measure([0, 1], [1, 0])  # Measuring qubits 0 and 1 to classical bits, reordered

# Simulate the protocol after step 6
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc_bob, simulator)
result = simulator.run(compiled_circuit).result()
counts = result.get_counts()
print("Measurement results after step 6:", counts)

# Step 7: Bob tests Alice's loyalty by performing the announced operations
qc_bob.x(1)  # Bob performs X on qubit 1 to recover state |0>

# Simulate the full protocol after step 7
compiled_circuit = transpile(qc_bob, simulator)
result = simulator.run(compiled_circuit).result()
counts = result.get_counts()
print("Measurement results after step 7 (qubit 1 only):", {key[-1]: val for key, val in counts.items() if key[0] == '1'})

# Output the results
#plot_histogram(counts)

Measurement results after step 6: {'00': 510, '10': 514}
Measurement results after step 7 (qubit 1 only): {'0': 479}


### Security Analysis

#### External Attack

without channel checking or reduced frequency, Eve will be able to illicitly eavesdrop on their messages<br>

Intercept-and-resend attack: <br>
will disturb qubits states <br>
each qubit will be disturbed with a probability of ¼ <br>

reads sequence, measures and resend sequence<br>

Entangling attack: <br>
intercepts transmission, prepares ancillary qubit $\ket E$ and perform unitary operation $U_e$

#### Internal Attack

Goal Alice: learn Bob's choices <br>
Goal Bob: learn content of Alice's messages <br>

#### Alice cheating strategy: <br>
<ins>No ability of entanglement:</ins>
- perform a single qubit gate {<i>I, X, Y, Z, H</i>}
- success probability: 25% or 29.3%
- attack always detected in protocol (which step?)
    - measurement in the incorrect basis can yield incorrect measurement results that nevertheless help Alice to determine Bob’s initial state



<ins>With ability of entanglement:</ins>
- perform two or more qubit gates
- Bob required to also have ability of entanglement to resist attacks
- attack will be detected
- success probability 25% or 29.3%
- teleportation attack: (in step 4)
    - instead of inputting secret message into qubit C creates bell state, gives qubit B to Bob to repace C, and keeps qubit A
    - Alice performs bell measurement on C and A
    - According to measurement results gives <i>I, X, Y, Z</i> as her secret message
    - Bob can then recover state $\ket0_B$ or $\ket+_B$ with apllying said gate

<ins>Bob also has entanglement ability (How Bob can counter the previous procedure):</ins> <br>
- prepare entangled state in $\frac{1}{\sqrt{2}}(\ket{00} + \ket{11})_{A1B1}$
- send qubit A1 to alice
- Alice performs an operation {<i>I, X, Y, Z</i>} on A1
- Bob can then measure qubits A1 and B1 with specific bases (IY basis and XZ basis) to check Alice's loyalty.

#### Bob cheating strategy: <br>
- prepare entangled qubits in the form $\ket{\phi^+} = \frac{1}{\sqrt{2}}(\ket{00} + \ket{11})_{12}$
- Bob can perfectly identify which operation Alice performed on qubit 1

### Efficency Analysis


Comparison of four protocols: Yang's, YYLSZ, YSW and the proposed protocol.

R = message bits that are received

Yang's protocol:
- uses B92 protocol
- four qubits for unambiguous key and only one transmission
- detection strategy with decoy qubits used
- $4 * R + 50$

YYYLSZ protocol:
- four qubits on average to obtain an unambiguous key using
its all-or-nothing QOT strategy i.e. p = 1/4
- two transmissions needed
- strategy is based on the law of large number and will consume a large number of qubits
- $4 * R + 2 * 50$


YSW protocol:
- four qubits for unambiguous key and one transmission
- not include loyalty testing
- Errors may be detected at the application level
- $4 * R + 50$

Proposed protocol:
- based directly on quantum resources and consumes one qubit for
each received bit
- requires two transmissions
- $R + 2 * 50$


<img src="QOTEfficencyComparison.png" width=500px height=130px/>

Quantum Resources for One Message Bit: Number of quantum resources consumed for each received bit, without decoy qubits <br>
Number of Transmissions: Number of transmissions for one sequence <br>
Decoy Qubits: Number of decoy qubits, considering the number of transmissions <br>
Total Cost: The total average quantum resource consumption for R received bits

<img src="QOTComparisonResult.png" width=400px height=300px/>